# BIA 6304 Text Mining, Fall 2017
## Assignment 2
### Stuent:  Leonardo Ji
### 9/3/2017

**T1.**  Using the data frame from Assignment 1 or another that you create with at least one column of text to be analyzed, instantiate a count vectorizer to create a bag of words. Use the default to convert to lower case.  Generate a vector space model. Try various combinations of vectorizer settings like we did in class: changing case, eliminating stop words, using min and max document frequency settings, choosing n-grams, etc. Try at least 3 different versions. 

![R-blogger news site](https://www.r-bloggers.com/wp-content/uploads/2016/04/R_02_2016-05-01.png "R-blogger")
I want to use R-blogger news site data from web scrapping technique used in the last class.

In [2]:
import pandas as pd   #we almost always need pandas because we like data frames
from bs4 import BeautifulSoup
import requests
import re
from sklearn.feature_extraction.text import CountVectorizer
import math
import pandas as pd
import numpy as np
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_colwidth', 150) #important for getting all the text

In [4]:
# R-Blogger news
page = requests.get('https://www.r-bloggers.com')

soup = BeautifulSoup(page.text, "html5lib")
#print(type(soup))
#print (soup.prettify)

In [45]:
BSobj = soup.find_all("div",{"class":"twopost"})

headlines = {}
for item in BSobj:
    date = item.find("div",{"class":"date"}).get_text()
    link = item.a["href"]
    author = item.find("a",{"rel":"author"}).get_text()
    excerpt = item.find("p",{"class":"excerpt"}).get_text().replace('\n', '')
    headlines[item.a.get_text()] = {}
    headlines[item.a.get_text()]["category"] = "Post"
    headlines[item.a.get_text()]["author"] = author
    headlines[item.a.get_text()]["date"] = date
    headlines[item.a.get_text()]["link"] = link
    headlines[item.a.get_text()]["excerpt"] = excerpt
    
#print(headlines)

cnndf = pd.DataFrame.from_dict(headlines,orient="index")
#print(cnndf.shape)
cnndf.reset_index(inplace=True)
cnndf.columns = ['Headline', 'Category', 'author', 'Date', 'link', 'excerpt']
textStr = cnndf['excerpt'].values
cnndf.head()

,Headline,Category,author,Date,link,excerpt
0,A guide to parallelism in R,Post,Florian Privé,"September 4, 2017",https://www.r-bloggers.com/a-guide-to-parallelism-in-r/,"In this post, I will talk about parallelism in R. This post will likely be biased towards the solutions I use. For example, I never use mcapply no..."
1,Analyzing Google Trends Data in R,Post,Jake Hoare,"September 4, 2017",https://www.r-bloggers.com/analyzing-google-trends-data-in-r-2/,"Google Trends shows the changes in the popularity of search terms over a given time (i.e., number of hits over time). It can be used to find searc..."
2,Beyond the basics of data.table: Smooth data exploration,Post,sindri,"September 5, 2017",https://www.r-bloggers.com/beyond-the-basics-of-data-table-smooth-data-exploration/,This exercise set provides practice using the fast and concise data.table package. If you are new to the syntax it is recommended that you start b...
3,Blocked Gibbs Sampling in R for Bayesian Multiple Linear Regression,Post,AO,"September 5, 2017",https://www.r-bloggers.com/blocked-gibbs-sampling-in-r-for-bayesian-multiple-linear-regression/,"In a previous post, I derived and coded a Gibbs sampler in R for estimating a simple linear regression. In this post, I will do the same for multi..."
4,Calculating Marginal Effects Exercises,Post,BC Mullins,"September 4, 2017",https://www.r-bloggers.com/calculating-marginal-effects-exercises/,A common experience for those in the social sciences migrating to R from SPSS or STATA is that some procedures that happened at the click of a but...



Sklean documentation [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

* In the first try I set binary to true, removed English stop word, and all other settings left in default settings. It returned 312 feature spaces.

In [6]:
# try changing a few parameters
cv1 = CountVectorizer(binary=True, stop_words = 'english') #define the transformation
cv1_dm = cv1.fit_transform(textStr) #apply the transformation

#print(type(cv1_dm))
print(cv1_dm.shape)
countVectororizerTable = pd.DataFrame(cv1_dm.toarray(), columns = cv1.get_feature_names())
countVectororizerTable.head()

(28, 312)


,000,100,11,14th,2017,abstractcontrol,accumulate,advocating,agreement,alexandria,...,viewed,vincent,virginia,visualization,vs,week,working,world,written,yang
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* In the second try I count total number of words (set binary to false), removed English stop words, used minimual document frequency of 1% to maximum document frequency of 50%. It returned 41 feature spaces.

In [14]:
cv2 = CountVectorizer(stop_words = 'english', min_df = .05, max_df =.5) #define the transformation
cv2_dm = cv2.fit_transform(textStr) #apply the transformation

#print(type(cv2_dm))
print(cv2_dm.shape)
countVectororizerTable = pd.DataFrame(cv2_dm.toarray(),columns = cv2.get_feature_names())
countVectororizerTable.head()

(28, 41)


,2017,analysis,announce,api,available,collection,conference,cran,data,developed,...,start,statistical,talk,time,use,used,users,using,variable,written
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,2,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* In the third try counted total number of words (set binary to false), removed English stop words, used minimual document frequency of 10% to maximum document frequency of 50%, used ngram range of one to two words. It returned 46 feature spaces.

In [13]:
cv3 = CountVectorizer(min_df = .05, max_df =.5, stop_words = "english", ngram_range = (1,2)) #define the transformation
cv3_dm = cv3.fit_transform(textStr) #apply the transformation
#print(type(cv3_dm))
print(cv3_dm.shape)

countVectororizerTable = pd.DataFrame(cv3_dm.toarray(), columns = cv3.get_feature_names())
countVectororizerTable.head()

(28, 46)


,2017,analysis,announce,api,available,available cran,collection,conference,cran,data,...,statistical,statistical analysis,talk,time,use,used,users,using,variable,written
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,2,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
names = cv3.get_feature_names()   #create list of feature names
#print(type(names), len(names))
count = np.sum(cv3_dm.toarray(), axis = 0) # add up feature counts 
#print(type(count), count.shape)
count2 = count.tolist()  # convert numpy array to list
#print(len(count2))
count_df = pd.DataFrame(count2, index = names, columns = ['total count']) # create a dataframe from the list
count_df.head() # notice entries are alphabetical

,total count
2017,2
analysis,3
announce,2
api,3
available,2


In [19]:
count_df.tail()

,total count
used,2
users,2
using,5
variable,2
written,2


**Q2.** Write a short description of what the vector space represents and how you can use it. Make sure your answer is no longer than three paragraphs, and should at minimum answer these questions:
*	How do the parameter settings affect the size of the feature space?
*	What parameter settings might be “best” for the question you have in mind? 
Hint:  if you want to show numbers, a table would work well here.  
Audience: technical – fellow data scientists or other technical staff.


The question I have in mind is what are some popular topics on R-blogger.  I can use count vectorizer table for this.  I collected twenty-eight blog posts from R-blogger site.  The dataset contains five columns: category, author, date, link, excerpt.  The excerpt column contains excerpt text that I can create a vector space model.  Vector space model convert text documents into numeric model such as a sparse matrix.  The Sklearn Learn library vector space function allow me to transform blog posts to a sparse matrix of documents (rows) and features/words (columns).  

The parameter settings affected the size of the feature space. 
A good setting is 0.1 for min document frequency and 0.5 for max document frequency, this removed words that is less than 10% of the documents or more than 50% of the documents.
We don't want to distinguish lower case vs upper case of the same terms. Another good setting is ngram from one to two words.  This would give us two words term such as "data science" in addition to single word terms "data" and "science". Another good setting is remove English stop words such as "a", "the", "to", "is"...etc. min_df, max_df, stop_words, lowercase all would reduce number of feature space come back from countVectorizerTable function.  The n_gram (1,2) setting would increase number of feature space because it considers both single word and double word terms. In the first try it returned 312 feature spaces, the second try returned 41 feature spaces, the third try returned 47 feature spaces.
###### Table 1. Run Results
| Runs                                                                  |  Feature Spaces               |
| --------------------------------------------------------------------- |:-----------------------------:|
| binary=True, stop_words = 'english'                                   |    312                        |
| stop_words = 'english', min_df = .05, max_df =.5                      |     41                        |  
| min_df = .05, max_df =.5, stop_words = "english", ngram_range = (1,2) |     46                        |

###### Table 2. Parameters Inputs
| Parameters    | Value         |  Meaning                                                                  |
| :------------:|:-------------:|:-------------------------------------------------------------------------:|
| min_df        | 0.1           |  Ignore terms that have a document frequency less than 0.1                |
| max_df        | 0.5           |  Ignore terms that have a document frequency more than 0.5                |
| stop_words    | english       |  Remove english stop words                                                |
| ngram_range   | (1,2)         |  Only allow 1 or 2 words terms                                            |
| binary        | False         |  Default setting to return integer count of the terms instead of binary   |
| lowercase     | True          |  Default setting to convert all characters to lower cases                 |
| encoding      | UTF8          |  Default setting to UTF8 encoding for the text                            |
 

**T3.** Repeat the above using the Tfidf vectorizer. Use a variety of parameter settings. Try at least 3 different versions. 

Sklearn documentation [TfidVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

* In the first try I did not use tf_idf weight, did not normalize value, used english stop words and set maximum features to 20. It returned 20 feature spaces.

In [153]:
# How do the weights work here?
tfidf1 = TfidfVectorizer(use_idf=False, norm=None, stop_words = "english", max_features = 20) #define the transformation
tf1_dm = tfidf1.fit_transform(textStr) #apply the transformation
print(tf1_dm.shape)
tfidfVectororizerTable = pd.DataFrame(tf1_dm.toarray(),columns = tfidf1.get_feature_names())
tfidfVectororizerTable.head()

(28, 20)


,__,analysis,data,dplyr,know,large,learning,library,numfocus,package,packageversion,practical,recently,science,sparklyr,statistical,time,use,using,version
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* In the second try I used tf_idf weight, did not normalize value, used english stop words and set maximum features to 20. It returned 20 feature spaces.

In [27]:
tfidf2 = TfidfVectorizer(use_idf=True, norm=None, stop_words = "english", max_features = 20) #define the transformation
tf2_dm = tfidf2.fit_transform(textStr) #apply the transformation
print(tf2_dm.shape)
tfidfVectororizerTable = pd.DataFrame(tf2_dm.toarray(),columns = tfidf2.get_feature_names())
tfidfVectororizerTable.head()

(28, 20)


,analysis,api,collection,conference,cran,data,dplyr,earl,idea,new,package,post,recently,regression,set,statistical,talk,time,use,using
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,4.842771,0.0,0.000000,0.000000,0.0,3.268684,0.000000,5.151073,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,6.537367,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,1.802346,0.0,0.0,0.0,2.981001,2.287854,0.000000,0.0,0.000000,5.962003,0.0,0.000000,0.000000,0.000000,2.757858
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,4.842771,0.0,3.268684,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


* In the third try I use tf_idf weight, did not normalize value, used english stop words, set min_df to 5%, set max_df to 50%, ngram 1 to 2. It returned 46 feature spaces.

In [43]:
tfidf3 = TfidfVectorizer(use_idf=True, norm=None, stop_words = "english", min_df = 0.05, max_df = 0.5, ngram_range = (1,2)) #define the transformation
tf3_dm = tfidf3.fit_transform(textStr) #apply the transformation
print(tf3_dm.shape)
tfidfVectororizerTable = pd.DataFrame(tf3_dm.toarray(), columns = tfidf3.get_feature_names())
tfidfVectororizerTable.head()

(28, 46)


,2017,analysis,announce,api,available,available cran,collection,conference,cran,data,...,statistical,statistical analysis,talk,time,use,used,users,using,variable,written
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,3.268684,0.000000,5.151073,0.000000,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,6.537367,0.000000,3.268684,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.802346,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,2.757858,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0


**Q3.** Write a short description of what the weights mean and how they might affect your analysis. Make sure your answer is no longer than two paragraphs, and should at minimum answer these questions:
* Does using weights make sense for your question?  If so, why?  If not, why not?
Audience: technical – fellow data scientists or other technical staff.

Document frequency df(j) shows the number of document containing word j. 
Inverse document frequency idf(j) reverses scale by applying logarithmically scale on it.  If word j is in many documents then it is less important, if word j appears in a few documents then it is unique and more important.

$$idf(j) = log(\frac{N}{df(j)})$$
![idf](formula1.png "idf")

Term frequency tf(j) of term j is the raw count of term j in the document. The term frequency inverse-document-frequency tf_idf weighting is calculated by multiply tf(j) and idf(j).

$$tf\_idf(j) = tf(j) ∗ idf(j)$$
![tf_idf](formula2.png "tf_idf")

Setting this function's flag "use_idf" to true returns tf_idf weighting.  The tf_idf weighting is used to compute a score for the term.  The bigger the score the more important the term's value of the documents.  
From this math foruma we can see:
* If a term only appears in one document for many times then the weight will be the highest.
* If a term appears only a few times or the document appears in a lot of document then the weight will be lower.
* If a term appears in all documents equally then the idf weight value would be zero.

**In this case my question is which of these posts I can read on regression.**  Using tf_idf weight make sense because I want to find a good post focused on "regression" out of all other posts on the R-blogger site.  If "regression" is mentioned in one post but not others, then it should have the highest weight on the term "regression". Yes, the document's weight of the term "regression" is high at 6.53 on one post, therefore the "Permutation Theory In Action"  is the post I need to read to learn more about how to use regression in R compare to all other posts on the R-blogger site.

Here is the blog post I should read, and its tf_idf weight of the terms from the result.

In [50]:
cnndf.loc[[16]]

,Headline,Category,author,Date,link,excerpt
16,Permutation Theory In Action,Post,John Mount,"September 2, 2017",https://www.r-bloggers.com/permutation-theory-in-action/,While working on a large client project using Sparklyr and multinomial regression we recently ran into a problem: Apache Spark chooses the order o...


In [48]:
print(tfidfVectororizerTable.loc[16])

2017                    0.000000
analysis                0.000000
announce                0.000000
api                     0.000000
available               0.000000
available cran          0.000000
collection              0.000000
conference              0.000000
cran                    0.000000
data                    0.000000
data science            0.000000
developed               0.000000
effective               0.000000
excited                 0.000000
excited announce        0.000000
high                    0.000000
idea                    0.000000
interface               0.000000
large                   3.268684
learning                0.000000
model                   0.000000
new                     0.000000
order                   3.268684
package                 0.000000
people                  0.000000
post                    0.000000
previous                0.000000
previous post           0.000000
process                 0.000000
provides                0.000000
reading   

**Q4.** Besides emojis and encodings that we discussed in class, what other changes might you consider making to the text, if any?  What effect would you expect this to have? If you wouldn’t make any other changes, explain why not. 
Audience: general – management or non-technical staff.

I would consider do the following:
* A lot of these blog posts have example code in it.  I would like to exclude the R language key words such as "package", "library", "version"...etc.  Remove them because the example code common keywords are not useful for understanding the question.
* The term such as "use" and "using" can be grouped into the same column by perform Stemming. One feature/column for the term "use". 
* Excludes verbs such as "need", "open", "open", "ran".  These verbs do not help understand the question I try to ask. Remove them from the table.
* Year 2017 is not the same as 2017 in total count. We need to distinguish them.
* Remove comma from numbers.  For example, convert 1,000 to 1000. '000' should not be a separate token.
* A run from last week showed special character "-" is separate out from "2-D" and "3-D".  It would treat "2D" and "3D" as a single token or remove "-" from "2-D" and "3-D".
* For multiple words term like "statistical analysis" if single word term "statistical" and "analysis" also exist with the same frequency then we know multiple words term is all we need.   

These will make the feature space terms more meaningful, and reduce the number of feature space terms that are too general.  